# new method to reflector.


first to design the operation of permutation, here use the 32-bit register, 16-cell, 2-block, for the permutation. we want let the result is the sequence operation of the shift & or & and, and have the operation register. the start register is r0, the operation temp register is r1, the result register is r14

In [162]:
# aes permutation
permutation = [0, 1, 2, 3, 7, 4, 5, 6, 10, 11, 8, 9, 13, 14, 15, 12]
# transform to sequence operation
sequence = []


def PPO(sequence, temp, origin, dest, mask, shift):
    sequence.append(("and", temp, origin, hex(mask)))
    sequence.append(("ror", temp, temp, shift))
    sequence.append(("or", dest, dest, temp))


for i in range(len(permutation)):
    # first need to mask to r1,
    # then self-shift r1, there have the ror operation, so all the shift direction is right,
    # if the shift is left, need to use the length 32 sub the shift value.
    # then or operation to r14
    mask = 2 << ((15 - i) * 2)
    distance = i - permutation.index(i)
    shift = 2 * (distance) if distance >= 0 else 2 * (distance + 16)
    PPO(sequence, "r1", "r0", "r14", mask, shift)
print(sequence)
print(len(sequence))
# here is not optimal sequence, so need to optimize the sequence

[('and', 'r1', 'r0', '0x80000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x8000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x800000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x200000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x80000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000'), ('ror', 'r1', 'r1', 6), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x8000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x800'), ('ror', 'r1', 'r1', 4), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x200'), ('ror', 'r1', 'r1', 4), ('or', '

one pop operation is including the and, ror, or operation, that format `[('and', 'r1', 'r0', '0x80000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1')]`

if only the mask value is not same, then can merge the operation, merge is xor the mask value.

In [163]:
def merge_sequence(sequence):
    # Group operations by 3 (one POP = and+ror+or)
    pops = [sequence[i : i + 3] for i in range(0, len(sequence), 3)]
    merged = []
    processed = set()

    for i, pop1 in enumerate(pops):
        if i in processed:
            continue

        mask1 = int(pop1[0][3], 16)  # Convert hex mask to int
        shift1 = int(pop1[1][3])
        current_group = [pop1]

        # Find pops with different masks that can be merged
        for j, pop2 in enumerate(pops[i + 1 :], i + 1):
            if j in processed:
                continue

            mask2 = int(pop2[0][3], 16)
            shift2 = int(pop2[1][3])
            # Check if masks are different
            if mask1 != mask2 and shift1 == shift2:
                # Merge masks using XOR
                mask1 ^= mask2
                current_group.append(pop2)
                processed.add(j)

        if len(current_group) > 1:
            # Create merged operation
            merged.extend(
                [
                    ("and", "r1", "r0", hex(mask1)),
                    ("ror", "r1", "r1", current_group[0][1][3]),
                    ("or", "r14", "r14", "r1"),
                ]
            )
        else:
            # Keep original operation
            merged.extend(pop1)

    return merged


merged = merge_sequence(sequence)
print(merged)
print(len(merged))

[('and', 'r1', 'r0', '0xaa000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa80000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000'), ('ror', 'r1', 'r1', 6), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa00'), ('ror', 'r1', 'r1', 4), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x80'), ('ror', 'r1', 'r1', 26), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2a'), ('ror', 'r1', 'r1', 2), ('or', 'r14', 'r14', 'r1')]
21


In [164]:
class PPO:
    def __init__(self, temp, origin, dest, mask, shift):
        self.temp = temp
        self.origin = origin
        self.dest = dest
        self.mask = mask
        self.shift = shift

    def __eq__(self, value: "PPO") -> bool:
        return (
            self.temp == value.temp
            and self.origin == value.origin
            and self.dest == value.dest
            and self.mask == value.mask
            and self.shift == value.shift
        )

    def toList(self):
        """Convert PPO operation to sequence list"""
        return [
            ("and", self.temp, self.origin, hex(self.mask)),
            ("ror", self.temp, self.temp, self.shift),
            ("or", self.dest, self.dest, self.temp),
        ]

    @staticmethod
    def merge(ppo_list):
        """Merge PPOs with different masks using XOR, when the all the other parameters are the same"""
        merged = []
        processed = set()

        for i, ppo1 in enumerate(ppo_list):
            if i in processed:
                continue

            mask = ppo1.mask
            current_group = [ppo1]

            for j, ppo2 in enumerate(ppo_list[i + 1 :], i + 1):
                if j in processed:
                    continue

                if (
                    mask != ppo2.mask
                    and ppo1.shift == ppo2.shift
                    and ppo1.temp == ppo2.temp
                    and ppo1.origin == ppo2.origin
                    and ppo1.dest == ppo2.dest
                ):
                    mask ^= ppo2.mask
                    current_group.append(ppo2)
                    processed.add(j)

            if len(current_group) > 1:
                merged.append(
                    PPO(
                        current_group[0].temp,
                        current_group[0].origin,
                        current_group[0].dest,
                        mask,
                        current_group[0].shift,
                    )
                )
            else:
                merged.append(ppo1)

        return merged


# aes permutation
permutation = [0, 1, 2, 3, 7, 4, 5, 6, 10, 11, 8, 9, 13, 14, 15, 12]
# transform to sequence operation
sequence = []
for i in range(len(permutation)):
    # first need to mask to r1,
    # then self-shift r1, there have the ror operation, so all the shift direction is right,
    # if the shift is left, need to use the length 32 sub the shift value.
    # then or operation to r14
    mask = 2 << ((15 - i) * 2)
    distance = i - permutation.index(i)
    shift = 2 * (distance) if distance >= 0 else 2 * (distance + 16)
    sequence.append(PPO("r1", "r0", "r14", mask, shift))

sequence = PPO.merge(sequence)
merged = []
for ppo in sequence:
    merged.extend(ppo.toList())
print(merged)
print(len(merged))

[('and', 'r1', 'r0', '0xaa000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa80000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000'), ('ror', 'r1', 'r1', 6), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa00'), ('ror', 'r1', 'r1', 4), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x80'), ('ror', 'r1', 'r1', 26), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2a'), ('ror', 'r1', 'r1', 2), ('or', 'r14', 'r14', 'r1')]
21


In [165]:
def print_sequence(sequence):
    """
    Print assembly-like sequence in standard format.
    Each operation on new line with proper indentation.
    """
    merged = []
    for ppo in sequence:
        if isinstance(ppo, PPO):
            merged.extend(ppo.toList())
        else:
            merged.append(ppo)
    print(merged)
    print(len(merged))


print_sequence(sequence)

[('and', 'r1', 'r0', '0xaa000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa80000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000'), ('ror', 'r1', 'r1', 6), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa00'), ('ror', 'r1', 'r1', 4), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x80'), ('ror', 'r1', 'r1', 26), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2a'), ('ror', 'r1', 'r1', 2), ('or', 'r14', 'r14', 'r1')]
21


In [166]:
def sort_sequence(sequence):
    """
    Sort sequence by operation type OPP.shift.
    """
    return sorted(sequence, key=lambda x: x.shift)


sequence = sort_sequence(sequence)
print_sequence(sequence)

[('and', 'r1', 'r0', '0xaa000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2a'), ('ror', 'r1', 'r1', 2), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa00'), ('ror', 'r1', 'r1', 4), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000'), ('ror', 'r1', 'r1', 6), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x80'), ('ror', 'r1', 'r1', 26), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa80000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1')]
21


In [167]:
p1 = PPO("r1", "r0", "r14", 0x80000000, 0)
p2 = PPO("r1", "r0", "r14", 0x80000000, 0)
print(p1 == p2)  # True

True


In [168]:
# here we need to split the PPO
# if the sequence[n] PPO0 is can split to PPO1 and PPO2, then split it, here condition need statisfied:
# 1. the PPO1 and PPO2 index on the sequence is lower than n, PPO1 < PPO2 < n.
# 2. the PPO1.shift+PPO2.shift = PPO0.shift
# 3. the PPO1.mask&(PPO0.mask) = 0, the PPO2.mask&(rotation_shift_right_32bit(PPO0.mask)) = 0
# if the condition is statisfied, then split the PPO0 to PPO1 and PPO2, return the pair of PPO1 and PPO2 index.
# else return None


def rotation_shift_right_32bit(value, shift):
    # Handle 32-bit right rotation
    return ((value >> shift) | (value << (32 - shift))) & 0xFFFFFFFF


def split(sequence: list["PPO"], n):
    """
    Try to split PPO0 at index n into PPO1 and PPO2 from earlier in sequence.

    Args:
        sequence: List of PPO objects
        n: Index of PPO0 to try splitting

    Returns:
        Tuple of (idx1, idx2) if split possible, None otherwise
    """

    ppo0 = sequence[n]
    mask0 = ppo0.mask
    shift0 = ppo0.shift

    # Check all possible pairs before index n
    for i in range(n):
        for j in range(i + 1, n):
            ppo1 = sequence[i]
            ppo2 = sequence[j]

            # Condition 1: PPO1 < PPO2 < n (already satisfied by loop structure)

            # Condition 2: shifts sum up
            if ppo1.shift + ppo2.shift != shift0:
                continue

            # Condition 3: mask conditions
            mask1 = ppo1.mask
            mask2 = ppo2.mask

            # Check first mask condition
            if mask1 & mask0 != 0:
                continue

            # Check second mask condition with rotation
            rotated_mask0 = rotation_shift_right_32bit(mask1 | mask0, ppo1.shift)
            if mask2 & rotated_mask0 != 0:
                continue

            # All conditions satisfied
            return (i, j)

    return None


for i in range(len(sequence)):
    split_idx = split(sequence, i)
    if split_idx:

        print(f"Split {i} into {split_idx}")
        break

Split 3 into (1, 2)


## OPO algorithm

**Input:** Set of pairs $ P(p,m) $, optimization index $ n $  
**Output:** Optimized set of pairs $ P' $

1. $ P' \leftarrow P $
2. if $ n = \text{Length}(P') $ then
3. &nbsp;&nbsp;&nbsp;&nbsp;return $ P' $
4. end if
5. $ P'_1 \leftarrow \text{OPO}(P', n + 1) $
6. if $ \text{Split}(P'[n]) \neq \emptyset $ then
7. &nbsp;&nbsp;&nbsp;&nbsp;(p1, m1), (p2, m2) $ \leftarrow \text{Split}(P'[n]) $
8. &nbsp;&nbsp;&nbsp;&nbsp;$ P' \leftarrow P' \cup \{(p1, m1), (p2, m2)\} $
9.  &nbsp;&nbsp;&nbsp;&nbsp;Delete $ P'[n] $
10. &nbsp;&nbsp;&nbsp;&nbsp;$ P'_2 \leftarrow \text{OPO}(P', n) $
11. end if
12. return $ \text{Better}(P'_1, P'_2) $

In [172]:
temp_register_number = 0


def OPO(P: list[PPO], n: int) -> list[PPO]:
    """
    Optimize permutation operations using the OPO algorithm.

    Args:
        P: List of PPO objects
        n: Current optimization index

    Returns:
        Optimized list of PPO objects
    """
    # 1. Make copy of input sequence
    # true copy, not reference
    P_prime = []
    for i in range(len(P)):
        P_prime.append(PPO(P[i].temp, P[i].origin, P[i].dest, P[i].mask, P[i].shift))

    # 2-4. Base case: reached end of sequence
    if n == len(P_prime):
        return P_prime

    # 5. Recursive optimization of remaining sequence
    P_prime1 = OPO(P_prime, n + 1)

    # 6. Try splitting current operation
    split_result = split(P_prime, n)
    if split_result is not None:
        # 7-9. Get split operations and update sequence
        i, j = split_result
        p0 = P_prime[n]
        p1, p2 = P_prime[i], P_prime[j]
        global temp_register_number

        p1.mask |= p0.mask
        is_add = 0
        # here temp register have optime space, when the temp register not depend on the other PPO, then can reuse it.
        if p1.dest == "r14":
            p1.dest = f"d{temp_register_number}"
            temp_register_number += 1
            is_add = 1
        m1 = rotation_shift_right_32bit(p0.mask, p1.shift)
        p2.mask |= m1
        p2.temp = p2.temp + f"|{p1.dest}"

        # Create new sequence with split operations
        P_prime2 = []
        for k in range(len(P_prime)):
            P_prime2.append(
                PPO(
                    P_prime[k].temp,
                    P_prime[k].origin,
                    P_prime[k].dest,
                    P_prime[k].mask,
                    P_prime[k].shift,
                )
            )
        P_prime2.pop(n)  # Delete original operation

        # 10. Recursive optimization with split sequence
        P_prime2 = OPO(P_prime2, n)

        if is_add:
            temp_register_number -= 1

        # 12. Return better sequence
        return better(P_prime1, P_prime2)

    # If no split possible, return sequence from step 5
    return P_prime1


def better(P1: list[PPO], P2: list[PPO]) -> list[PPO]:
    """
    Compare two sequences and return the better one.
    Currently uses sequence length as metric.
    """
    return P1 if len(P1) <= len(P2) else P2


# Example usage:
optimized_sequence = OPO(sequence, 0)
print_sequence(optimized_sequence)

[('and', 'r1', 'r0', '0xaa000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2a02a'), ('ror', 'r1', 'r1', 2), ('or', 'd0', 'd0', 'r1'), ('and', 'r1|d0', 'r0', '0x8a00'), ('ror', 'r1|d0', 'r1|d0', 4), ('or', 'r14', 'r14', 'r1|d0'), ('and', 'r1|d0', 'r0', '0x2880'), ('ror', 'r1|d0', 'r1|d0', 26), ('or', 'r14', 'r14', 'r1|d0'), ('and', 'r1', 'r0', '0xa80000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1')]
15


In [173]:
print("sequence")
for i in sequence:
    print(i.temp, i.origin, i.dest, hex(i.mask), i.shift)
print("optimized_sequence")
for i in optimized_sequence:
    print(i.temp, i.origin, i.dest, hex(i.mask), i.shift)

sequence
r1 r0 r14 0xaa000000 0
r1 r0 r14 0x2a 2
r1 r0 r14 0xa00 4
r1 r0 r14 0x20000 6
r1 r0 r14 0x80 26
r1 r0 r14 0xa000 28
r1 r0 r14 0xa80000 30
optimized_sequence
r1 r0 r14 0xaa000000 0
r1 r0 d0 0x2a02a 2
r1|d0 r0 r14 0x8a00 4
r1|d0 r0 r14 0x2880 26
r1 r0 r14 0xa80000 30
